In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
from sqlalchemy import create_engine
print("pandas:", pd.__version__, "numpy:", np.__version__)

pandas: 2.2.2 numpy: 2.0.2


In [5]:
from google.colab import files
uploaded = files.upload()

!ls -l

Saving claims_data.csv to claims_data.csv
Saving food_listings_data.csv to food_listings_data.csv
Saving providers_data.csv to providers_data.csv
Saving receivers_data.csv to receivers_data.csv
total 284
-rw-r--r-- 1 root root  37225 Aug 21 10:13 claims_data.csv
-rw-r--r-- 1 root root  72777 Aug 21 10:13 food_listings_data.csv
-rw-r--r-- 1 root root 112505 Aug 21 10:13 providers_data.csv
-rw-r--r-- 1 root root  55920 Aug 21 10:13 receivers_data.csv
drwxr-xr-x 1 root root   4096 Aug 19 13:38 sample_data


In [6]:
DATA_DIR = Path("/content/data")
DATA_DIR.mkdir(exist_ok=True)

for fname in uploaded.keys():
  src = Path("/content") / fname
  dst = DATA_DIR / fname
  if src.exists():
    src.rename(dst)
    print(f"Moved {fname} -> {dst}")
  else:
    print(f"WARNING: {src} not found (maybe it was already moved).")

print("\nFiles in data folder:")
print(list(DATA_DIR.iterdir()))

Moved claims_data.csv -> /content/data/claims_data.csv
Moved food_listings_data.csv -> /content/data/food_listings_data.csv
Moved providers_data.csv -> /content/data/providers_data.csv
Moved receivers_data.csv -> /content/data/receivers_data.csv

Files in data folder:
[PosixPath('/content/data/claims_data.csv'), PosixPath('/content/data/food_listings_data.csv'), PosixPath('/content/data/providers_data.csv'), PosixPath('/content/data/receivers_data.csv')]


In [7]:
providers_fp = DATA_DIR / "providers_data.csv"
receivers_fp = DATA_DIR / "receivers_data.csv"
food_fp = DATA_DIR / "food_listings_data.csv"
claims_fp = DATA_DIR / "claims_data.csv"

providers = pd.read_csv(providers_fp)
receivers = pd.read_csv(receivers_fp)
food_listings = pd.read_csv(food_fp)
claims = pd.read_csv(claims_fp)

print("providers", providers.shape)
print("receivers", receivers.shape)
print("food_listings", food_listings.shape)
print("claims", claims.shape)

providers.head()

providers (1000, 6)
receivers (1000, 5)
food_listings (1000, 9)
claims (1000, 5)


,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [8]:
for df in [providers, receivers, food_listings, claims]:
  df.columns = [c.strip() for c in df.columns]

print("providers columns:", providers.columns.tolist())
print("food_listings columns:", food_listings.columns.tolist())
print("claims column:", claims.columns.tolist())

providers columns: ['Provider_ID', 'Name', 'Type', 'Address', 'City', 'Contact']
food_listings columns: ['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID', 'Provider_Type', 'Location', 'Food_Type', 'Meal_Type']
claims column: ['Claim_ID', 'Food_ID', 'Receiver_ID', 'Status', 'Timestamp']


In [9]:
def to_int_col(df, col):
  if col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype('Int64')

to_int_col(providers, "Provider_ID")
to_int_col(receivers, "Receiver_ID")
to_int_col(food_listings, "Food_ID")
to_int_col(food_listings, "Provider_ID")
to_int_col(food_listings, "Quantity")
to_int_col(claims, "Claim_ID")
to_int_col(claims, "Food_ID")
to_int_col(claims, "Receiver_ID")

if "Expiry_Date" in food_listings.columns:
  food_listings["Expiry_Date"] = pd.to_datetime(food_listings["Expiry_Date"], errors="coerce").dt.date
if "Timestamp" in claims.columns:
  claims["Timestamp"] = pd.to_datetime(claims["Timestamp"], errors="coerce")

text_cols = ["Type", "Provider_Type", "City", "Food_Type", "Meal_Type", "Status", "Location", "Name", "Address", "Contact"]
for df in [providers, receivers, food_listings, claims]:
  for c in text_cols:
    if c in df.columns:
      df[c] = df[c].astype("string").str.strip()

print("providers dtypes:\n", providers.dtypes)
print("\nfood_listings dtypes:\n", food_listings.dtypes)
print("\nclaims dtypes:\n", claims.dtypes)


providers dtypes:
 Provider_ID             Int64
Name           string[python]
Type           string[python]
Address        string[python]
City           string[python]
Contact        string[python]
dtype: object

food_listings dtypes:
 Food_ID                   Int64
Food_Name                object
Quantity                  Int64
Expiry_Date              object
Provider_ID               Int64
Provider_Type    string[python]
Location         string[python]
Food_Type        string[python]
Meal_Type        string[python]
dtype: object

claims dtypes:
 Claim_ID                Int64
Food_ID                 Int64
Receiver_ID             Int64
Status         string[python]
Timestamp      datetime64[ns]
dtype: object


In [10]:
print("---- Missing counts ----")
print("providers missing:\n", providers.isna().sum())
print("\nfood_listings missing:\n", food_listings.isna().sum())
print("\nclaims missing:\n", claims.isna().sum())

print("\n---- Duplicate PK checks ----")
if "Provider_ID" in providers.columns:
    print("providers duplicate Provider_ID:", providers.duplicated(subset=["Provider_ID"]).sum())
if "Receiver_ID" in receivers.columns:
    print("receivers duplicate Receiver_ID:", receivers.duplicated(subset=["Receiver_ID"]).sum())
if "Food_ID" in food_listings.columns:
    print("food_listings duplicate Food_ID:", food_listings.duplicated(subset=["Food_ID"]).sum())
if "Claim_ID" in claims.columns:
    print("claims duplicate Claim_ID:", claims.duplicated(subset=["Claim_ID"]).sum())

if "Quantity" in food_listings.columns:
    print("\nQuantity min/max:", food_listings["Quantity"].min(), "/", food_listings["Quantity"].max())

for c in ["City","Food_Type","Meal_Type","Provider_Type","Status"]:
    if c in food_listings.columns:
        print(f"\nUnique '{c}' in food_listings (sample up to 20):")
        print(food_listings[c].dropna().unique()[:20])


---- Missing counts ----
providers missing:
 Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64

food_listings missing:
 Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64

claims missing:
 Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64

---- Duplicate PK checks ----
providers duplicate Provider_ID: 0
receivers duplicate Receiver_ID: 0
food_listings duplicate Food_ID: 0
claims duplicate Claim_ID: 0

Quantity min/max: 1 / 50

Unique 'Food_Type' in food_listings (sample up to 20):
<StringArray>
['Non-Vegetarian', 'Vegan', 'Vegetarian']
Length: 3, dtype: string

Unique 'Meal_Type' in food_listings (sample up to 20):
<StringArray>
['Breakfast', 'Dinner', 'Lunch', 'Snacks']
Length: 4, dtype: string

Unique 'Provider_Type' in food_listings (sample 

In [11]:
if "Food_ID" in food_listings.columns and "Provider_ID" in food_listings.columns:
    before = len(food_listings)
    food_listings = food_listings.dropna(subset=["Food_ID","Provider_ID"])
    print("Dropped", before - len(food_listings), "rows from food_listings missing keys")

if "Contact" in providers.columns:
    providers["Contact"] = providers["Contact"].fillna("Unknown")

if "Food_Type" in food_listings.columns:
    mapping = {"veg":"Vegetarian","Veg":"Vegetarian","vegetarian":"Vegetarian",
               "non-veg":"Non-Vegetarian","non_veg":"Non-Vegetarian","vegan":"Vegan"}
    food_listings["Food_Type"] = food_listings["Food_Type"].replace(mapping).astype("string")

if "Quantity" in food_listings.columns:
    bad = (food_listings["Quantity"].isna()) | (food_listings["Quantity"] <= 0)
    print("Rows with bad quantities:", bad.sum())
    food_listings = food_listings[~bad]

print("\nAfter cleaning: shapes")
print("providers:", providers.shape)
print("food_listings:", food_listings.shape)
print("claims:", claims.shape)


Dropped 0 rows from food_listings missing keys
Rows with bad quantities: 0

After cleaning: shapes
providers: (1000, 6)
food_listings: (1000, 9)
claims: (1000, 5)


In [12]:
providers.to_csv(DATA_DIR / "providers_cleaned.csv", index=False)
receivers.to_csv(DATA_DIR / "receivers_cleaned.csv", index=False)
food_listings.to_csv(DATA_DIR / "food_listings_cleaned.csv", index=False)
claims.to_csv(DATA_DIR / "claims_cleaned.csv", index=False)

print("Saved cleaned CSVs to", DATA_DIR)
print(list(DATA_DIR.glob("*cleaned.csv")))


Saved cleaned CSVs to /content/data
[PosixPath('/content/data/providers_cleaned.csv'), PosixPath('/content/data/receivers_cleaned.csv'), PosixPath('/content/data/food_listings_cleaned.csv'), PosixPath('/content/data/claims_cleaned.csv')]


In [17]:
!rm -f /content/food_wastage.db


In [18]:
from pathlib import Path
from sqlalchemy import create_engine
import sqlalchemy

db_fp = Path("/content/food_wastage.db")
engine = create_engine(f"sqlite:///{db_fp}", echo=False)

# Use engine.begin() to ensure each write closes properly
with engine.begin() as conn:
    providers.to_sql("providers", con=conn, if_exists="replace", index=False)
    receivers.to_sql("receivers", con=conn, if_exists="replace", index=False)
    food_listings.to_sql("food_listings", con=conn, if_exists="replace", index=False)
    claims.to_sql("claims", con=conn, if_exists="replace", index=False)

# Verify
with engine.connect() as conn:
    tables = conn.execute(sqlalchemy.text("SELECT name FROM sqlite_master WHERE type='table';")).fetchall()
    print("Tables in DB:", tables)

Tables in DB: [('providers',), ('receivers',), ('food_listings',), ('claims',)]


In [19]:
q1 = "SELECT City, COUNT(*) as provider_count FROM providers GROUP BY City;"
df1 = pd.read_sql(q1, con=engine)
print(df1)


                 City  provider_count
0            Adambury               1
1           Adamsview               1
2          Adamsville               1
3        Aguirreville               1
4    Alexanderchester               1
..                ...             ...
958       Wrightville               1
959         Yatesside               1
960      Youngchester               1
961      Zimmermanton               1
962    Zimmermanville               1

[963 rows x 2 columns]


In [20]:
# Run all 13 project queries

queries = {
    "Q1. Providers per city": """
        SELECT City, COUNT(*) AS provider_count
        FROM providers
        GROUP BY City
        ORDER BY provider_count DESC;
    """,

    "Q1b. Receivers per city": """
        SELECT City, COUNT(*) AS receiver_count
        FROM receivers
        GROUP BY City
        ORDER BY receiver_count DESC;
    """,

    "Q2. Provider type contributing most food": """
        SELECT Provider_Type, COUNT(*) AS total_listings
        FROM food_listings
        GROUP BY Provider_Type
        ORDER BY total_listings DESC;
    """,

    "Q3. Provider contact info in specific city": f"""
        SELECT Name, Contact, Address, City
        FROM providers
        WHERE City = 'Delhi';  -- 🔹 Change 'Delhi' as needed
    """,

    "Q4. Top receivers by claims": """
        SELECT r.Name, COUNT(c.Claim_ID) AS total_claims
        FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        GROUP BY r.Name
        ORDER BY total_claims DESC
        LIMIT 10;
    """,

    "Q5. Total quantity of food available": """
        SELECT SUM(Quantity) AS total_food_available FROM food_listings;
    """,

    "Q6. City with highest number of food listings": """
        SELECT Location AS City, COUNT(*) AS total_listings
        FROM food_listings
        GROUP BY Location
        ORDER BY total_listings DESC
        LIMIT 1;
    """,

    "Q7. Most commonly available food types": """
        SELECT Food_Type, COUNT(*) AS count_food
        FROM food_listings
        GROUP BY Food_Type
        ORDER BY count_food DESC;
    """,

    "Q8. Claims made per food item": """
        SELECT f.Food_Name, COUNT(c.Claim_ID) AS claims_count
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY f.Food_Name
        ORDER BY claims_count DESC;
    """,

    "Q9. Provider with highest successful claims": """
        SELECT p.Name, COUNT(c.Claim_ID) AS successful_claims
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Name
        ORDER BY successful_claims DESC
        LIMIT 1;
    """,

    "Q10. Claim status distribution (%)": """
        SELECT Status,
               COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS percentage
        FROM claims
        GROUP BY Status;
    """,

    "Q11. Average quantity claimed per receiver": """
        SELECT r.Name, AVG(f.Quantity) AS avg_quantity_claimed
        FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY r.Name
        ORDER BY avg_quantity_claimed DESC;
    """,

    "Q12. Most claimed meal type": """
        SELECT f.Meal_Type, COUNT(c.Claim_ID) AS total_claims
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY f.Meal_Type
        ORDER BY total_claims DESC;
    """,

    "Q13. Total quantity donated by each provider": """
        SELECT p.Name, SUM(f.Quantity) AS total_donated
        FROM food_listings f
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        GROUP BY p.Name
        ORDER BY total_donated DESC;
    """
}

# Run and display all queries
for title, sql in queries.items():
    print("="*80)
    print(title)
    print("="*80)
    display(pd.read_sql(sql, con=engine))


Q1. Providers per city


,City,provider_count
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


Q1b. Receivers per city


,City,receiver_count
0,New Christopher,3
1,Williamsfort,2
2,West Richard,2
3,West James,2
4,Stevenmouth,2
...,...,...
961,Alexanderbury,1
962,Aguilarstad,1
963,Aguilarbury,1
964,Adamland,1


Q2. Provider type contributing most food


...

,Name,avg_quantity_claimed
0,Thomas Villanueva,50.0
1,Peggy Knight,50.0
2,Nancy Silva,50.0
3,Nancy Jones,50.0
4,Lisa Pitts,50.0
...,...,...
615,Kenneth Barnes,2.0
616,John Reynolds,2.0
617,Corey Rodriguez,2.0
618,Joshua Hooper,1.0


Q12. Most claimed meal type


,Meal_Type,total_claims
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


Q13. Total quantity donated by each provider


,Name,total_donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Reyes and Sons,1
624,Norris-Townsend,1
625,Mcgee PLC,1
626,"Martinez, Armstrong and Carroll",1


In [21]:
from google.colab import files
files.download("/content/food_wastage.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>